In [35]:
import boto3


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

athena_client = boto_session.client(service_name="athena", region_name=region)

default_s3_bucket_name = "s3://sagemaker-us-west-2-846634201516/athena_notebook"

In [36]:
query = "SELECT id, sum(payment) from feature04 group by id;"

query_response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": "customer"},
    ResultConfiguration={
        "OutputLocation": default_s3_bucket_name,
    },
)

print(query)

SELECT id, sum(payment) from feature04 group by id;


In [37]:
import pandas as pd

S3_BUCKET_NAME = "sagemaker-us-west-2-846634201516"
S3_OUTPUT_DIRECTORY = "athena_notebook"
temp_file_location: str = "./athena_query_results.csv"
s3_client = boto3.client(
    service_name="s3",
    region_name=region,
)

s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
df = pd.read_csv(temp_file_location)

In [38]:
import sagemaker
from sagemaker.session import Session


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [39]:
# read dynamodb feature
dynamodb = boto3.client('dynamodb')
record = dynamodb.get_item(TableName='customer', Key={'id':{'S':'1'}})

In [61]:
children=record['Item']['children']['N']
children

'2'

In [41]:
!pip install pandas

In [46]:
# Offline feature from athena
import pandas as pd
import json
churn = pd.read_csv("athena_query_results.csv")

In [59]:
id = churn['id'].values[0]
print(id)

1


In [60]:
payments = churn["_col1"].values[0]
payments

4

In [45]:
from sagemaker.session import Session
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
boto_session = boto3.Session(region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [71]:
# write fo feature store
from sagemaker.feature_store.feature_group import FeatureGroup
user_feature_group = FeatureGroup(
    name="customer", sagemaker_session=sagemaker_session
)
data = [[str(id), int(children), payments, '2023-07-01T14:59:55.711Z']]
df = pd.DataFrame(data, columns=['id', 'children', 'payments', 'time'])
df

,id,children,payments,time
0,1,2,4,2023-07-01T14:59:55.711Z


In [ ]:
user_feature_group.ingest(data_frame=df, max_workers=3, wait=True)

In [72]:
# read from feature store
record_identifier_value = str(1)

record = featurestore_runtime.get_record(
    FeatureGroupName="customer",
    RecordIdentifierValueAsString=record_identifier_value,
)
record

{'ResponseMetadata': {'RequestId': '62e50a72-1dc0-4d9c-9d55-216f1ff30c32',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '62e50a72-1dc0-4d9c-9d55-216f1ff30c32',
   'content-type': 'application/json',
   'content-length': '330',
   'date': 'Sun, 24 Mar 2024 05:15:26 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'children', 'ValueAsString': '2'},
  {'FeatureName': 'payments', 'ValueAsString': '4'},
  {'FeatureName': 'id', 'ValueAsString': '1'},
  {'FeatureName': 'time', 'ValueAsString': '2023-07-01T14:59:55.711Z'}]}

In [81]:
record['Record'][1]['ValueAsString']

'4'